In [5]:
import pandas as pd
from database.market import Market
import matplotlib.pyplot as plt
from processor.processor import Processor as p

In [3]:
market = Market()

In [4]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [6]:
prices = p.column_date_processing(prices)

E:\projects\longshot_venv\longshot\processor\processor.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["week"] = data["date"].dt.week


In [10]:
quarterlies = prices.groupby(["year","quarter","ticker"]).mean().reset_index()

In [11]:
tickers = quarterlies["ticker"].unique()

In [12]:
analysis = []
for ticker in tickers:
    ticker_data = quarterlies[(quarterlies["ticker"]==ticker) & (quarterlies["year"]==2022)]
    ticker_data.sort_values("quarter",ascending=True,inplace=True)
    ticker_data["previous"] = ticker_data["adjclose"].shift(-1)
    ticker_data["delta"] = (ticker_data["adjclose"] - ticker_data["previous"]) / ticker_data["adjclose"]
    analysis.append(ticker_data)

C:\Users\eric\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\eric\AppData\Local\Temp/ipykernel_13232/3497463333.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["previous"] = ticker_data["adjclose"].shift(-1)
C:\Users\eric\AppData\Local\Temp/ipykernel_13232/3497463333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [23]:
a = pd.concat(analysis)

In [16]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [24]:
a = a.merge(sp500.rename(columns={"Symbol":"ticker"})[["GICS Sector","ticker","Security"]],on="ticker",how="left")

In [27]:
final = []
for quarter in range(1,4):
    quarter_performance = a[a["quarter"]==quarter][["quarter","GICS Sector","ticker","Security","delta"]].sort_values(["quarter","delta"],ascending=False).dropna().iloc[:10]
    final.append(quarter_performance)

In [28]:
pd.concat(final)

,quarter,GICS Sector,ticker,Security,delta
1159,1,Communication Services,NFLX,Netflix,0.467476
1883,1,Consumer Discretionary,ETSY,Etsy,0.385888
88,1,Health Care,ALGN,Align Technology,0.368779
1895,1,Information Technology,PYPL,PayPal,0.348140
1711,1,Communication Services,WBD,Warner Bros. Discovery,0.329798
1619,1,Consumer Discretionary,UAA,Under Armour (Class A),0.315873
1431,1,Financials,SBNY,Signature Bank,0.308940
1863,1,Consumer Discretionary,CZR,Caesars Entertainment,0.306475
855,1,Information Technology,IPGP,IPG Photonics,0.281125
623,1,Consumer Discretionary,F,Ford Motor Company,0.275135
